### Q2

In [ ]:
# Basic numerics
import pandas as pd
import scipy.stats as st
import numpy as np
import scipy as sp

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

# Statistical modelling tools
import statsmodels.api as sm
from scipy.stats import spearmanr,chi2_contingency
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,roc_auc_score
from sklearn.metrics import confusion_matrix


# Load data from CSV file
df = pd.read_csv('MavenRail.csv')


In [ ]:
#solve missing values in the dataframe
def solve_missing_values(df):
    # Delete rows with missing values in 'Departure' and 'Scheduled.Arrival'
    df = df[df['Departure'].notnull()]
    df = df[df['Scheduled.Arrival'].notnull()]

    # Fill missing# define the function to handle the missing values values for 'Railcard' with 'No Railcard'
    if 'Railcard' in df.columns and df['Railcard'].isnull().any():
        df['Railcard'] = df['Railcard'].fillna('No Railcard')

    # Handle missing values in 'Actual.Arrival'
    if 'Journey.Status' in df.columns and 'Actual.Arrival' in df.columns:
        # Fill with invalid time for 'Cancelled' status
        df.loc[df['Journey.Status'] == 'Cancelled', 'Actual.Arrival'] = pd.Timestamp('01/01/1900')

        # Fill with 'Scheduled.Arrival' for 'On Time' status
        df.loc[
            (df['Journey.Status'] == 'On Time') & (df['Actual.Arrival'].isnull()), 
            'Actual.Arrival'
        ] = df['Scheduled.Arrival']

    # Fill missing values for 'Reason.for.Delay' with 'No Delay' for 'On Time' status
    if 'Journey.Status' in df.columns and 'Reason.for.Delay' in df.columns:
        df.loc[df['Journey.Status'] == 'On Time', 'Reason.for.Delay'] = 'No Delay'

    return df

In [ ]:
# Evaluating the performance of models
def evaluate_model(model, X, Y, test_sizes):
    #store proportions of each test set
    results = []

    for test_size in test_sizes:
        # Split the dataset
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
        
        # Train the model
        model.fit(x_train, y_train)
        
        # Make predictions
        y_pred = model.predict(x_test)
        y_proba = model.predict_proba(x_test)[:, 1]  # Probability predictions for AUC
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)
        auc = roc_auc_score(y_test, y_proba)
        
        # Append results
        results.append({
            'test_size': test_size,
            'accuracy': accuracy,
            'precision_yes': report['1']['precision'],  # Precision for the "Yes" class
            'recall_yes': report['1']['recall'],        # Recall for the "Yes" class
            'f1_score_yes': report['1']['f1-score'],   # F1-score for the "Yes" class
            'auc': auc                                 # AUC score
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
# prepare_data function is used to one-hot encode the specified features and extract the target variable from the given dataframe.
def prepare_data(df, features_to_encode, target_variable='Refund.Request'):
    # One-hot encode the specified features
    X = pd.get_dummies(df[features_to_encode], columns=features_to_encode, drop_first=True)
    # Extract the target variable
    Y = df[target_variable]
    return X, Y


In [ ]:
# Plotting confusion matrices with percentages
def plot_confusion_matrix(model, x_train, y_train, x_test, y_test, title='Confusion Matrix Visualization with Percentages'):
    # Train the model
    model.fit(x_train, y_train)

    # Make predictions
    y_pred = model.predict(x_test)

    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()  # Unpack values
    total = np.sum(cm)  # Total number of samples

    # Calculate percentages
    percentages = [tn / total * 100, fp / total * 100, fn / total * 100, tp / total * 100]

    # Plot the confusion matrix with percentages
    fig, ax = plt.subplots(figsize=(6, 6))

    # Create a color-coded grid for the confusion matrix
    categories = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
    values = [tn, fp, fn, tp]
    colors = ['lightgreen', 'orange', 'pink', 'lightblue']

    # Plot the value and category of each element in the confusion matrix
    for i, (value, pct, category, color) in enumerate(zip(values, percentages, categories, colors)):
        row, col = divmod(i, 2)  # Determine row and column
        ax.add_patch(plt.Rectangle((col, row), 1, 1, color=color, alpha=0.7)) # Add a rectangular block
        ax.text(
            col + 0.5, row + 0.6, f'{category}', ha='center', va='center', fontsize=12 # Display category name
        )
        ax.text(
            col + 0.5, row + 0.4, f'{value} ({pct:.2f}%)', ha='center', va='center', fontsize=12 # Display values and percentages
        )

    # Adjust plot aesthetics
    ax.set_xlim(0, 2) # Limited x-axis range
    ax.set_ylim(0, 2) # Limited y-axis range
    ax.set_xticks([0.5, 1.5]) # Set the x-axis scale
    ax.set_xticklabels(['Predicted Negative', 'Predicted Positive']) # Set the x-axis label
    ax.set_yticks([0.5, 1.5]) # Set the y-axis scale
    ax.set_yticklabels(['Actual Positive', 'Actual Negative']) # Set the y-axis label
    ax.set_xlabel('Prediction') # Set the x-axis title
    ax.set_ylabel('Actual') # Set the y-axis title
    ax.set_title(title) # Set the graph title
    plt.gca().invert_yaxis()  # Flip y-axis to align with traditional confusion matrix view
    plt.show()# Display the graph


In [ ]:
# Define the datetime columns to process
datetime_columns = ['Departure', 'Scheduled.Arrival', 'Actual.Arrival']

# Convert these columns to datetime format
for col in datetime_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce') # Convert to datetime64


In [ ]:
#Print metadata information before data cleansing
print("Metadata Before Cleaning:")
print(df.info())


In [ ]:
# Print missing value statistics
print("\nMissing value statistics:")
print(df.isnull().sum())

In [ ]:
# Handling missing values ​​in the dataset
df = solve_missing_values(df)

In [ ]:
# Print metadata information after data cleansing
print("Metadata After Cleaning:")
print(df.info())

In [ ]:
print("\nMissing value statistics after cleaning:")
print(df.isnull().sum())

In [ ]:
#Chaneg the outlier
# Check if Departure is later than Scheduled.Arrival and puls 1 day
df.loc[df['Departure'] > df['Scheduled.Arrival'], 'Scheduled.Arrival'] += pd.Timedelta(days=1)

In [ ]:
# counting the number of trips for each route
route_num = df.groupby(['Departure.Station', 'Arrival.Station']).size().reset_index(name='Count')
# Sort by number of trips from high to low
route_num = route_num.sort_values(by='Count', ascending=False)
# print result
print("Number of trips for each route:")
print(route_num) # print top 5 rows

In [ ]:
# Calculation of travel time in minutes
duration = (df['Actual.Arrival'] - df['Departure']).dt.total_seconds() / 60

# Create a new DataFrame containing only the relevant columns
duration_df = pd.DataFrame({
    'Arrival.Station': df['Arrival.Station'],
    'Departure.Station':df['Departure.Station'],
    'Trip Duration': duration
})

# Sort trips by duration in descending order
duration_sorted = duration_df.sort_values('Trip Duration', ascending=False)

# print result
print(duration_sorted.head())


In [ ]:
# Calculation of travel time in minutes
duration = (df['Actual.Arrival'] - df['Departure']).dt.total_seconds() / 60

# Create a new DataFrame containing only the relevant columns
duration_df = pd.DataFrame({
    'Arrival.Station': df['Arrival.Station'],
    'Departure.Station': df['Departure.Station'],
    'Departure': df['Departure'],
    'Actual.Arrival': df['Actual.Arrival'],
    'Mean Trip Duration': duration
})

# Group by unique routes (Departure.Station and Arrival.Station) and calculate the mean duration
average_duration = duration_df.groupby(['Departure.Station', 'Arrival.Station'])['Mean Trip Duration'].mean().reset_index()

# Sort trips by mean duration in descending order
average_duration_sorted = average_duration.sort_values('Mean Trip Duration', ascending=False)

# Print the result
print(average_duration_sorted.head())


In [ ]:
# Plotting price distribution
x = df['Price']
g=sns.displot( x, kde=True, rug=False)
for ax in g.axes.flat:  
    ax.legend(labels=["KDE", "Ticket Price"])

# Set title and axis labels
plt.title("Price Distribution")
plt.xlabel("Price")
plt.ylabel("Frequency")
# Adjust the X-axis range and scale
plt.xlim(0, 270)  # Limit the X-axis range to 0 to 270
plt.xticks(range(0, 270, 50))  # Set the X-axis tick interval to 50


In [ ]:
# Create a figure with subplots (3 rows, 2 columns)
fig, axes = plt.subplots(3, 2, figsize=(18, 18))

# List of variables to plot and corresponding titles
variables = ['Journey.Status', 'Payment.Method', 'Ticket.Class', 
             'Refund.Request','Ticket.Type','Railcard']
titles = ['Journey Status Distribution', 'Payment Method Distribution', 
          'Ticket Class Distribution', 'Refund Request Distribution',
          'Ticket Type Distribution','Railcard Distribution' ]

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each variable and create a pie chart
for ax, var, title in zip(axes, variables, titles):
    # Count the occurrences of each category
    counts = df[var].value_counts()
    
    # Plot the pie chart in percentage
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90, 
           colors=sns.color_palette('pastel'), textprops={'fontsize': 19})
    # Set sub-figure title
    ax.set_title(title, fontsize=20)

# Remove unused subplots if the number of variables is less than 6
for ax in axes[len(variables):]:
    ax.remove()

# Adjust sub-graph layout to avoid overlap
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of occurrences of delay reasons
delay_reason_counts = df['Reason.for.Delay'].value_counts()

# Set the graphics size
plt.figure(figsize=(12, 8))

# Draw horizontal bar graphs
delay_reason_counts.plot(kind='barh', color='skyblue', figsize=(12, 8))

# Add chart titles and axis labels
plt.title('Counts of Delay Reasons', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Delay Reason', fontsize=14)
# Add grid lines
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Adjust layout to avoid content overlap
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of occurrences of each departure station
departure_counts = df['Departure.Station'].value_counts()
# Count the number of occurrences of each arrival station
arrival_counts = df['Arrival.Station'].value_counts()

# Combine the counts into a single DataFrame
combined_counts = pd.DataFrame({
    'Departure': departure_counts,
    'Arrival': arrival_counts
}).fillna(0)  # Fill NaN with 0 for stations that exist only in one category

# Sort by the total count (optional, for better visualization)
combined_counts['Total'] = combined_counts['Departure'] + combined_counts['Arrival']
combined_counts = combined_counts.sort_values('Total', ascending=False).drop(columns=['Total'])

# Setting the graphic size
plt.figure(figsize=(12, 10))

# Draw horizontal bar graphs
combined_counts.plot(kind='barh', stacked=False, figsize=(14, 10), color=['skyblue', 'orange'])

# Add chart titles and axis labels
plt.title('Counts of Departure and Arrival Stations', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Station', fontsize=14)
# Add a legend
plt.legend(title="Category", fontsize=12)
# Add gridlines
plt.grid(axis='x', linestyle='--', alpha=0.7)

#  Adjust the layout to avoid overlapping content
plt.tight_layout()
plt.show()


In [ ]:
# Display statistical summary for 'Price'
print("\nStatistics of Price:")
print(df['Price'].describe())

In [ ]:
# Map 'Yes'/'No' to 1/0 for Refund.Request
df['Refund.Request'] = df['Refund.Request'].map({'Yes': 1, 'No': 0})

# Iterate over the specified columns and calculate the mean Refund.Request
for i in ['Payment.Method', 'Railcard', 'Ticket.Class', 'Ticket.Type',  
          'Departure.Station', 'Arrival.Station', 'Journey.Status', 'Reason.for.Delay',
          'Price','Departure', 'Scheduled.Arrival', 'Actual.Arrival']:
    # Group by current column (i) and calculate the mean value of 'Refund.Request'
    print((df[[i, 'Refund.Request']].groupby(i, as_index=False).mean().sort_values(by='Refund.Request', ascending=False)).head(5))
    print('-'*10,'\n')


### Q3

In [ ]:
# calculate delay time
delay_time = (df['Actual.Arrival'] - df['Departure'])
# Convert delay time to minutes and add to new column 'DelayInMinutes'
df['DelayInMinutes'] = delay_time.dt.total_seconds() / 60
# Replace non-positive delay times with NaN
df['DelayInMinutes'] = df['DelayInMinutes'].where(df['DelayInMinutes']>0,np.nan)
# Print the delay time column after processing
print("\nDelay time in minutes：")
print(df[['Scheduled.Arrival', 'Actual.Arrival', 'DelayInMinutes']])

In [ ]:
# Count the number of NaN values in DelayInMinutes
nan_count = df['DelayInMinutes'].isnull().sum()
print("Number of NaN values in DelayInMinutes:", nan_count)

### Q4

In [ ]:
# Add a new column MediumPrice to mark if the ticket price is in the range (10 < Price <= 30)
df['MediumPrice'] = np.where((df['Price'] > 10) & (df['Price'] <= 30), 1, 0)

# Filter Journey.Status is not "On Time"
filtered_df = df[df['Journey.Status'] != 'On Time'].copy()

# Prepare data for logistic regression
X = filtered_df[['MediumPrice']]
Y = filtered_df['Refund.Request']

# Split data into train and test sets (80% for training, 20% for test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Use logistic regression model to train data
model = LogisticRegression()
model.fit(X_train, Y_train)

# Predict the target variable on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

# MediumPrice regression coefficients
beta1 = model.coef_[0][0]
print("Coefficient of MediumPrice:", beta1)

# Intercept of the model
beta0 = model.intercept_
print("Intercept:", beta0)

# Calculate the probability of a refund for a given fare
input_5 = pd.DataFrame([[0]], columns=['MediumPrice'])  # £5 (out of range)
input_25 = pd.DataFrame([[1]], columns=['MediumPrice'])  # £25 (in range)

# Use the model to predict probabilities
prob_5 = model.predict_proba(input_5)[0][1]
prob_25 = model.predict_proba(input_25)[0][1]

# Print the refund probability for a given ticket price
print(f"Probability of requesting a refund for £5 ticket: {prob_5:.2%}")
print(f"Probability of requesting a refund for £25 ticket: {prob_25:.2%}")


### Q5

In [ ]:
# Load data from CSV file
predicted = pd.read_csv('ToPredict.csv')

In [ ]:
#Print missing values
print("\nMissing value statistics:")
print(predicted.isnull().sum())

In [ ]:
# Use previously defined functions to handle missing values
predicted = solve_missing_values(predicted)

In [ ]:
# Print missing value statistics after cleaning
print("\nMissing value statistics after washing:")
print(predicted.isnull().sum())

In [ ]:
# Separate numeric and categorical variables
numeric_vars = df.select_dtypes(include=['int64']).columns
categorical_vars = df.select_dtypes(include=['object','datetime64[ns]','float64']).columns

# Initialisation result storage
spearman_results = [] # Storage Spearman correlation analysis results
chi_square_results = [] # Stored chi-square test results

# Calculate Spearman's correlation for numerical variables
for var in numeric_vars:
    if var != 'Refund.Request':  # Exclude the target variable itself
        correlation, p_value = spearmanr(df[var], df['Refund.Request'])
        spearman_results.append((var, correlation, p_value))

# Print Spearman correlation analysis results
print("Spearman correlation results (sorted by p-value):")
for var, correlation, p_value in spearman_results:
    print(f"{var}: correlation={correlation:.4f}, p-value={p_value:.4f}")

# Calculate Chi-square test for categorical variables
for var in categorical_vars:
    if var != 'Refund.Request':  # Exclude the target variable itself
        contingency_table = pd.crosstab(df[var], df['Refund.Request'])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        chi_square_results.append((var, chi2,p))

# Sort results by chi-square value in ascending ascending
chi_square_results = sorted(chi_square_results, key=lambda x: x[1])

# Print chi-square test results
print("\nChi-square test results (sorted by p-value):")
for var,chi2, p in chi_square_results:
    print(f"{var}: Chi-square = {chi2:.4f}, p-value={p}")

In [ ]:
# Define the feature columns to be used in the model
features = ['Payment.Method', 'DelayInMinutes', 'Journey.Status', 
            'Reason.for.Delay', 'Departure.Station', 'Arrival.Station']
#Returns the feature matrix X and the target variable Y
X, Y = prepare_data(df, features, 'Refund.Request')


In [ ]:
# Define the random forest model
random_forest_model = RandomForestClassifier(random_state=42, n_estimators=100)

# Define the test set ratio list
test_sizes = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# Evaluating the Random Forest Model
results_rf_x1 = evaluate_model(random_forest_model, X, Y, test_sizes)

# Printing evaluation results
print("Random Forest(Base Features):")
print(results_rf_x1)

In [ ]:
# Data split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, Y, test_size=0.25, random_state=42)

# Define the random forest mode
random_forest_model = RandomForestClassifier(random_state=42, n_estimators=100)

# Call the function that draws the confusion matrix
plot_confusion_matrix(random_forest_model, x_train1, y_train1, x_test1, y_test1, 
                      title='Random Forest Confusion Matrix,test size=0.25')


In [ ]:
# Initialise the logistic regression model
logistic_model = LogisticRegression(random_state=42)

# Define the test set ratio list
test_sizes = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

# Evaluation of logistic regression models
results_logistic = evaluate_model(logistic_model, X, Y, test_sizes)

# Printing evaluation results
print("Logistic Regression (Base Features):")
print(results_logistic)


In [ ]:
#Logistic regression with Ticket.Type feature
# Data preparation: add `Ticket.Type` feature
features2 = ['Payment.Method', 'DelayInMinutes', 'Journey.Status', 
             'Reason.for.Delay', 'Departure.Station', 'Arrival.Station', 'Ticket.Type']
#Generate feature matrix X2 and target variable Y
X2, Y2 = prepare_data(df, features2,'Refund.Request')

# Defining a logistic regression model
logistic_model2 = LogisticRegression(random_state=42)

# Evaluating Logistic Regression Model Performance
results_logistic_x2 = evaluate_model(logistic_model2, X2, Y2, test_sizes)

# Printing evaluation results
print("Logistic Regression (Features with Ticket.Type):")
print(results_logistic_x2)


In [ ]:
# Data preparation: Add `Price` feature
features3 = ['Payment.Method', 'DelayInMinutes', 'Journey.Status', 
             'Reason.for.Delay', 'Departure.Station', 'Arrival.Station','Price']
X3, Y3 = prepare_data(df, features3,'Refund.Request')

# Generate feature matrix X3 and target variable Y
logistic_model3 = LogisticRegression(random_state=42)

# Defining a logistic regression model
results_logistic_x3 = evaluate_model(logistic_model3, X3, Y3, test_sizes)

# Printing evaluation results
print("Logistic Regression (Features with Price):")
print(results_logistic_x3)



In [ ]:
# Data partitioning: Divide the data into training set and test set
x_train2, x_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.25, random_state=42)

# Define the logistic regression model
logistic_model = LogisticRegression(random_state=42)

# Call the function to draw the confusion matrix
plot_confusion_matrix(logistic_model, x_train2, y_train2, x_test2, y_test2, 
                      title='Logistic Regression Confusion Matrix,test size=0.25')


In [ ]:
# Convert 'Scheduled.Arrival' and 'Actual.Arrival' to date and time format
predicted['Scheduled.Arrival'] = pd.to_datetime(predicted['Scheduled.Arrival'], errors='coerce')
predicted['Actual.Arrival'] = pd.to_datetime(predicted['Actual.Arrival'], errors='coerce')

# Fill missing values ​​in the 'Railcard' column with 'No Railcard'
predicted['Railcard'] = predicted['Railcard'].fillna('No Railcard')

#Solve the missing values in the 'Actual.Arrival'
# Handle 'Cancelled' trips
predicted.loc[predicted['Journey.Status'] == 'Cancelled', 'Actual.Arrival'] = pd.NaT
# Handle 'On Time' journeys
predicted.loc[
    (predicted['Journey.Status'] == 'On Time') & (predicted['Actual.Arrival'].isnull()),
    'Actual.Arrival'
] = predicted['Scheduled.Arrival']

# Set Reason.for.Delay to 'No Delay' for 'On Time' journeys
predicted.loc[predicted['Journey.Status'] == 'On Time', 'Reason.for.Delay'] = 'No Delay'

# Add DelayInMinutes column
predicted['DelayInMinutes'] = (predicted['Actual.Arrival'] - predicted['Scheduled.Arrival']).dt.total_seconds() / 60
# Fill missing or NaN delay times with 0
predicted['DelayInMinutes'] = predicted['DelayInMinutes'].fillna(0)

# One-hot encode 'predicted' data
predicted_encoded = pd.get_dummies(predicted, columns=[
    'Payment.Method', 'Journey.Status', 'Reason.for.Delay', 
    'Departure.Station', 'Arrival.Station','DelayInMinutes'
], drop_first=True)

# Align 'predicted' data with training data
predicted_encoded = predicted_encoded.reindex(columns=X.columns, fill_value=0)

# Make predictions using the logistic regression model
logistic_predictions = logistic_model.predict(predicted_encoded)

# Convert predictions to probabilities
logistic_probabilities = logistic_model.predict_proba(predicted_encoded)[:, 1]

# Add predictions and probabilities to the original DataFrame
predicted['Logistic_Predictions'] = logistic_predictions
predicted['Logistic_Probabilities'] = logistic_probabilities

# Print results
print(predicted[['Payment.Method', 'Journey.Status', 'Reason.for.Delay', 
                 'Departure.Station', 'Arrival.Station', 'DelayInMinutes',
                 'Logistic_Predictions','Logistic_Probabilities']])

